In [ ]:
import pandas as pd
import numpy as np
import datetime
import category_encoders as ce
import sys

In [ ]:
# the direction to the main directory
direct = sys.path[0]

In [ ]:
# Read the Coolblue datafiles
traffic_data = pd.read_csv(direct+r'\Coolblue_datasets\traffic_data', parse_dates=['date_time'],
                                      index_col=['date_time'])
broadcasting_data = pd.read_csv(direct+r'\Coolblue_datasets\broadcasting_data', parse_dates=[['date', 'time']])


# For each case, the datapreparation has to be done individually------------------------------------------------------------
traffic_datasets = []

# first filter out the push notifications and bounces
# CASE 1:-------------------------------------------------------------------------------------------------------------------
traffic_datasets.append(traffic_data[(traffic_data['visit_source']!='push notification') & (traffic_data['bounces']!=1)])


# CASE 2:-------------------------------------------------------------------------------------------------------------------
traffic_datasets.append(traffic_data[(traffic_data['visit_source']!='push notification') &
                            (traffic_data['visit_source']!='paid search') &
                            (traffic_data['bounces']!=1)])

# CASE 3:-------------------------------------------------------------------------------------------------------------------
traffic_datasets.append(traffic_data[(traffic_data['visit_source']=='direct') &
                           (traffic_data['bounces']!=1)])

# CASE 4:-------------------------------------------------------------------------------------------------------------------
traffic_datasets.append(traffic_data[(traffic_data['visit_source']!='push notification') &
                            (traffic_data['visit_source']!='paid search') &
                            (traffic_data['visit_source']!='other') &
                            (traffic_data['bounces']!=1)])

In [ ]:
# Functions that are used to build datasets with the spotlift---------------------------------------------------------------

# There are some outliers that looks like they are actually caused by push notifications,-----------------------------------
# therefore, they should not be included
def filter_outliers(data):
    new = data
    new.loc[new['visits_index']>0.1, 'visits_index']=0
    return new

# adjust noon observations-------------------------------------------------------------------------------------------
# It appears that at 23.59h there is an unexplainable peak. It might be that this is the result of all unassigned visits of
# the day that therefore ends in the last minute. 23.59h will be the average of 23.58h and 00.00h, and the other visits 
# will be spread over the day
def adj_midnight(data, perc):
    start_day = data.index.min()
    end_day = data.index.max()
    data = data.reset_index()
    noons = data[data['date_time'].dt.time==datetime.time(0,0)]
    
    for idx, row in noons.iterrows():
        curr = row['visits_index']
        if curr !=0:
            if idx ==0:
                average = data.loc[idx+1, 'visits_index']
                diff = curr - average
                if diff/curr >0.1:
                    data.loc[idx, 'visits_index'] = average     #Only correct first evaluation
            else:
                average = (data.loc[idx-1, 'visits_index'] + data.loc[idx+1, 'visits_index'])/2
                diff = curr - average
                if diff/curr >perc:
                    #Add 1/1439 of the difference to each observation of the day and change noon in average
                    data.loc[idx-1439:idx-1, 'visits_index']= data.loc[idx-1439:idx-1, 'visits_index'] +diff/1439
                    data.loc[idx, 'visits_index'] = average
    return data.set_index('date_time')

# Main function to build the dependend variable-----------------------------------------------------------------------------
def prep_data(data,broadcasting,country,medium,time_effect):
    relevant = data.copy()
    dates = pd.date_range(start=relevant.index.min(), end =relevant.index.max(), freq='min')
    dates = pd.DataFrame({'visits_index': 0}, index = dates)
    dates.index.name = 'date_time'
    
    relevant_nl = relevant[(relevant['country']== country) & (relevant['medium']== medium)]
    
    # Filter outliers if medium is app
    if medium =='app':
        relevant_nl = filter_outliers(relevant_nl)

    visits_nl = relevant_nl.groupby('date_time').sum()['visits_index']
    visits_nl = pd.DataFrame(visits_nl)

    final_nl = pd.merge(dates, visits_nl, left_on = 'date_time', right_on = 'date_time', how='left').sum(axis=1)
    final_nl = pd.DataFrame(final_nl, columns = ['visits_index'])
    final_nl['ads'] = 0

    # The length of spots are a minimum of 30seconds and maximum of 45 seconds, which means less than a minute
    # Adjustment: accuracy of date_time to minutes, to match them with 'final'
    broad_nl = broadcasting[broadcasting['country']==country]

    ads_starts_nl = broad_nl['date_time'].astype('datetime64[m]').sort_values().drop_duplicates() #all ads starting times
    ads_dates = dates.rename(columns={'visits_index': 'ads'}) #all timestamps

    last_min = visits_nl.index.max()
    boundary = last_min-pd.Timedelta(minutes=time_effect)

    # loop over all starting times and add the time_effect, store these in ads_dates
    for ad in ads_starts_nl:
        if ad<boundary:
            effect = pd.date_range(start=ad, periods = time_effect, freq='min')
            ads_dates.loc[effect, 'ads'] = 1
        else:
            effect = pd.date_range(start=ad, periods = ((last_min-ad).total_seconds()/60+1), freq='min')
            ads_dates.loc[effect, 'ads'] = 1
            break

    final_nl['ads'] = ads_dates['ads']
    final_nl = adj_midnight(final_nl,0)
    
    #Add Column with numbers of ads that started that minute
    broad_nl['date_time'] = broad_nl['date_time'].astype('datetime64[m]')
    ads_starts = broad_nl.groupby('date_time')['date_time'].count()
    final_nl['start_ad'] = 0
    final_nl.loc[list(ads_starts.index), 'start_ad'] = ads_starts.values
    
    return final_nl

In [ ]:
# NEXT THE X-MATRIX WILL BE CREATED
# There are two ways, one way that will aggregate features if multiple adds are aggregated: build_X()
# the second way is just the features per advertisement: build_X_blocks()

In [ ]:
# X_matrix based on grouped adds---------------------------------------------------------------------------------------------
def build_X(data, country,time_eff):
    time_effect=time_eff
    broad_nl = data[data['country']==country]
    broad_nl['date_time'] = broad_nl['date_time'].astype('datetime64[m]')
    broad_nl = broad_nl.sort_values('date_time', ignore_index = True)
    # add columns
    broad_nl['pos_program'] = (broad_nl['program_before']==broad_nl['program_after']).astype(int)
    broad_nl['weekend'] = (broad_nl['date_time'].dt.weekday.values>3).astype(int)
    broad_nl['prime'] = broad_nl['date_time'].dt.hour.between(18,23).values.astype(int)
    broad_nl['prime_18']= broad_nl['date_time'].dt.time.between(datetime.time(18,15,0),
                                                                   datetime.time(22, 15, 0)).values.astype(int)
    broad_nl['prime_16']= broad_nl['date_time'].dt.time.between(datetime.time(16,0,0),
                                                                   datetime.time(18, 15, 0)).values.astype(int)
    broad_nl['prime_22']= broad_nl['date_time'].dt.time.between(datetime.time(22,15,0),
                                                                   datetime.time(23, 59, 0)).values.astype(int)
    broad_nl['post_eff'] = broad_nl['date_time'] + pd.Timedelta(minutes=time_effect-1)
    # remove columns
    broad_nl = broad_nl.drop(['program_before', 
                              'program_after', 
                              'program_category_before', 
                              'program_category_after',
                              'country'], axis = 1)
    ads_starts_nl = broad_nl['date_time'].drop_duplicates() #all ads starting times
    # rename columns
    broad_nl = broad_nl.rename(columns = {'length_of_spot': 'duration',
                               'position_in_break': 'pos_break',
                               'product_category':'prod_cat'})
    
    # group ads within the post-effect
    broad_nl['ad_group'] = 1
    gr = 1
    for ad in range(len(broad_nl)-1):
        time_delta = broad_nl.loc[ad+1]['date_time']-broad_nl.loc[ad]['date_time']
        if time_delta<pd.Timedelta(minutes=time_effect):
            broad_nl.loc[ad+1, 'ad_group'] = gr
            continue
        gr = gr+1
        broad_nl.loc[ad+1, 'ad_group'] = gr 

    groups = broad_nl.groupby('ad_group')
    mult = groups.size().index[(groups.size()>1).values]

    # The Netherlands does not have categories for the positions in the break------------------------------------
    if country == 'Netherlands':
        broad_nl['pos_break'] = broad_nl['pos_break'].astype(int)
        # Talpa TV
        broad_nl.loc[(broad_nl['operator']=='Talpa TV') & (broad_nl['pos_break']<=2), 'pos_break'] +=1
        
        broad_nl.loc[(broad_nl['pos_break']>=3) & (broad_nl['pos_break']<=97), 'pos_break'] = 'Any Other Position'
        broad_nl.loc[(broad_nl['pos_break']==1), 'pos_break'] = 'First Position'
        broad_nl.loc[(broad_nl['pos_break']==2), 'pos_break'] = 'Second Position'
        broad_nl.loc[(broad_nl['pos_break']==98), 'pos_break'] = 'Before Last Position'
        broad_nl.loc[(broad_nl['pos_break']==99), 'pos_break'] = 'Last Position'
        
    X_nl = groups.first()
    encoder = ce.OneHotEncoder(cols=['operator', 'pos_break', 'duration', 'prod_cat', 'channel'],drop_invariant=True, 
                               use_cat_names=True).fit(broad_nl.drop('ad_group', axis=1))
    X_nl = encoder.transform(X_nl)

    # Adjust the data for multiple ads at the same time
    for key in mult:
        x = groups.get_group(key)
        x = encoder.transform(x)
        cols = x.drop(['date_time', 'post_eff'], axis=1).columns
        X_nl.loc[key, cols] = x.sum()[cols]
        X_nl.loc[key,'post_eff'] = x['post_eff'].max()

    X_nl['num_ads'] = groups.size().values
    return X_nl

In [ ]:
# X_matrix based on each advertisement individually-----------------------------------------------------------------------------
def build_X_blocks(data, country,time_eff):
    time_effect=time_eff
    broad_nl = data[data['country']==country]
    broad_nl['date_time'] = broad_nl['date_time'].astype('datetime64[m]')
    broad_nl = broad_nl.sort_values('date_time', ignore_index = True)
    # add columns
    broad_nl['pos_program'] = (broad_nl['program_before']==broad_nl['program_after']).astype(int)
    broad_nl['weekend'] = (broad_nl['date_time'].dt.weekday.values>3).astype(int)
    broad_nl['prime'] = broad_nl['date_time'].dt.hour.between(18,23).values.astype(int)
    broad_nl['prime_18']= broad_nl['date_time'].dt.time.between(datetime.time(18,15,0),
                                                                   datetime.time(22, 14, 0)).values.astype(int)
    broad_nl['prime_16']= broad_nl['date_time'].dt.time.between(datetime.time(16,0,0),
                                                                   datetime.time(18, 14, 0)).values.astype(int)
    broad_nl['prime_22']= broad_nl['date_time'].dt.time.between(datetime.time(22,15,0),
                                                                   datetime.time(23, 59, 0)).values.astype(int)
    broad_nl['post_eff'] = broad_nl['date_time'] + pd.Timedelta(minutes=time_effect-1)
    # remove columns
    broad_nl = broad_nl.drop(['program_before', 
                              'program_after', 
                              'program_category_before', 
                              'program_category_after',
                              'country'], axis = 1)
    ads_starts_nl = broad_nl['date_time'].drop_duplicates() #all ads starting times
    # rename columns
    broad_nl = broad_nl.rename(columns = {'length_of_spot': 'duration',
                               'position_in_break': 'pos_break',
                               'product_category':'prod_cat'})
    
    # group ads within the post-effect
    broad_nl['ad_group'] = 1
    gr = 1
    for ad in range(len(broad_nl)-1):
        time_delta = broad_nl.loc[ad+1]['date_time']-broad_nl.loc[ad]['date_time']
        if time_delta<pd.Timedelta(minutes=time_effect):
            broad_nl.loc[ad+1, 'ad_group'] = gr
            continue
        gr = gr+1
        broad_nl.loc[ad+1, 'ad_group'] = gr 

    # The Netherlands does not have categories for the positions in the break------------------------------------
    if country == 'Netherlands':
        broad_nl['pos_break'] = broad_nl['pos_break'].astype(int)
        # Talpa TV
        broad_nl.loc[(broad_nl['operator']=='Talpa TV') & (broad_nl['pos_break']<=2), 'pos_break'] +=1
        
        broad_nl.loc[(broad_nl['pos_break']>=3) & (broad_nl['pos_break']<=97), 'pos_break'] = 'Any Other Position'
        broad_nl.loc[(broad_nl['pos_break']==1), 'pos_break'] = 'First Position'
        broad_nl.loc[(broad_nl['pos_break']==2), 'pos_break'] = 'Second Position'
        broad_nl.loc[(broad_nl['pos_break']==98), 'pos_break'] = 'Before Last Position'
        broad_nl.loc[(broad_nl['pos_break']==99), 'pos_break'] = 'Last Position'
        
    X_nl = broad_nl
    encoder = ce.OneHotEncoder(cols=['operator', 'pos_break', 'duration', 'prod_cat', 'channel'],drop_invariant=True, 
                               use_cat_names=True).fit(broad_nl)
    X_nl = encoder.transform(X_nl)
        
    return X_nl

In [ ]:
# Main code to build all different X_matrices that are needed

# Start with the advertisement groups for different time_effects: [1,2,...,10]
for time_effect in range(1,11):
    X_nl = build_X(broadcasting_data, 'Netherlands', time_effect)
    X_nl.to_csv(direct+r'\X_matrix\X_nl_'+ str(time_effect) + 'min.csv', index_label='ad_group')
    
    X_be = build_X(broadcasting_data, 'Belgium', time_effect)
    X_be.to_csv(direct+r'\X_matrix\X_be_'+ str(time_effect) + 'min.csv', index_label='ad_group')
    
# 4 minutes seems best, so for the blocks method, we only make the dataset based on 4 minutes
time_effect = 4
X_nl = build_X_blocks(broadcasting_data, 'Netherlands', time_effect)
X_nl.to_csv(direct+r'\X_matrix\X_nl_'+ str(time_effect) + 'min_blocks.csv', index_label='ad_group')

X_be = build_X_blocks(broadcasting_data, 'Belgium', time_effect)
X_be.to_csv(direct+r'\X_matrix\X_be_'+ str(time_effect) + 'min_blocks.csv', index_label='ad_group')

In [ ]:
# Create the different datasets of the visits_index per case:
time_effect = 4
for idx, traffic_data in enumerate(traffic_datasets):
    case = idx+1
    
    nl_app = prep_data(traffic_data, broadcasting_data, 'Netherlands', 'app', time_effect)
    nl_web = prep_data(traffic_data, broadcasting_data, 'Netherlands', 'website', time_effect)
    be_app = prep_data(traffic_data, broadcasting_data, 'Belgium', 'app', time_effect)
    be_web = prep_data(traffic_data, broadcasting_data, 'Belgium', 'website', time_effect)

    nl_app.to_csv(direct+r'\visits\nl_app_case' +str(case) + '_4min.csv')
    nl_web.to_csv(direct+r'\visits\nl_web_case' +str(case) + '_4min.csv')
    be_app.to_csv(direct+r'\visits\be_app_case' +str(case) + '_4min.csv')
    be_web.to_csv(direct+r'\visits\be_web_case' +str(case) + '_4min.csv')